In [1]:
import pandas as pd 
import numpy as np 
import re 
from functools import reduce  # Importa reduce de functools para combinar datos
from wordcloud import WordCloud  

In [4]:
df = pd.read_csv(r'Dataset_houses_for_sale.csv')
df.head()

,primary_photo,last_update_date,source,tags,permalink,other_listings,list_date,open_houses,description,branding,...,search_promotions,location,rent_to_own,products,virtual_tours,community,listing_id,price_reduced_amount,primary,status
0,{'href': 'https://ap.rdcpix.com/9a76341d092543...,2023-09-28T20:08:52Z,"{'agents': [{'office_name': None}], 'id': 'ALA...","['forced_air', 'big_yard', 'farm', 'ranch', 't...",1423-Atkinson-Dr_Anchorage_AK_99504_M72066-31892,"{'rdc': [{'listing_id': '2960080461', 'listing...",2023-09-28T20:09:21Z,NaN,"{'sqft': 720, 'baths_consolidated': '1', 'lot_...","[{'name': 'EXP Realty South (Branch Office)', ...",...,NaN,"{'address': {'postal_code': '99504', 'state': ...",NaN,"{'products': ['core.agent', 'co_broke'], 'bran...",NaN,NaN,2.960080e+09,NaN,True,for_sale
1,{'href': 'https://ap.rdcpix.com/3ee6f071ed96cf...,2023-09-28T19:10:32Z,"{'agents': [{'office_name': None}], 'id': 'ALA...","['community_outdoor_space', 'den_or_office', '...",17610-Birchtree-St_Chugiak_AK_99567_M92453-51489,"{'rdc': [{'listing_id': '2960077863', 'listing...",2023-09-28T19:11:18Z,NaN,"{'sqft': 1860, 'baths_consolidated': '2', 'lot...","[{'name': 'RE MAX Dynamic Properties', 'photo'...",...,NaN,"{'address': {'postal_code': '99567', 'state': ...",NaN,"{'products': ['core.agent', 'core.broker', 'li...",NaN,NaN,2.960078e+09,NaN,True,for_sale
2,{'href': 'https://ap.rdcpix.com/f4e61be59e935a...,2023-09-28T19:08:29Z,{'agents': [{'office_name': 'Neue Realty Group...,"['garage_1_or_more', 'garage_2_or_more', 'big_...",1115-Gav-Way_Anchorage_AK_99504_M71616-89165,"{'rdc': [{'listing_id': '2960077191', 'listing...",2023-09-28T18:58:18Z,NaN,"{'sqft': 1876, 'baths_consolidated': '2', 'lot...","[{'name': 'Real Brokers Anchorage', 'photo': N...",...,NaN,"{'address': {'postal_code': '99504', 'state': ...",NaN,"{'products': ['core.agent', 'co_broke'], 'bran...",NaN,NaN,2.960077e+09,NaN,True,for_sale
3,{'href': 'https://ap.rdcpix.com/5b9f5fa3f8a71d...,2023-09-28T17:33:39Z,"{'agents': [{'office_name': None}, {'office_na...","['city_view', 'community_outdoor_space', 'fire...",6478-Fairweather-Dr-64_Anchorage_AK_99518_M862...,"{'rdc': [{'listing_id': '2960073322', 'listing...",2023-09-28T17:34:48Z,NaN,"{'sqft': 1372, 'baths_consolidated': '2', 'lot...","[{'name': 'Real Estate Brokers Of Alaska', 'ph...",...,NaN,"{'address': {'postal_code': '99518', 'state': ...",NaN,"{'products': ['core.agent', 'co_broke'], 'bran...",NaN,NaN,2.960073e+09,NaN,True,for_sale
4,{'href': 'https://ap.rdcpix.com/d131cb6c1aaded...,2023-09-28T18:22:48Z,"{'agents': [{'office_name': None}], 'id': 'ALA...","['community_outdoor_space', 'corner_lot', 'den...",1604-Jefferson-Ave_Anchorage_AK_99517_M87448-2...,"{'rdc': [{'listing_id': '2960072612', 'listing...",2023-09-28T17:18:23Z,NaN,"{'sqft': 1344, 'baths_consolidated': '2', 'lot...","[{'name': 'Herrington and Company, LLC', 'phot...",...,NaN,"{'address': {'postal_code': '99517', 'state': ...",NaN,"{'products': ['core.agent', 'core.broker', 'co...",NaN,NaN,2.960073e+09,NaN,True,for_sale


In [ ]:
#leer el archivo
houses_for_sale = pd.read_csv('Dataset_houses_for_sale.csv')

#columnas para eliminar con datos que no son relevantes
columns_to_drop=['source','permalink','other_listings','open_houses','branding','coming_soon_date','matterport','search_promotions',
                 'rent_to_own','products','virtual_tours','community','price_reduced_amount']

#eliminar las columas
houses_for_sale.drop(columns=columns_to_drop,inplace=True)

In [5]:
def data_processor(df, columnas, columna_target):
    """
    Procesa datos de columnas con representaciones de diccionarios.

    Parameters:
        df (DataFrame): DataFrame que contiene los datos.
        columnas (list): Lista de nombres de columnas.
        columna_target (str): Nombre de la columna objetivo.

    Returns:
        DataFrame: DataFrame procesado.
    """
    lista_dics_datos = []  # Inicializa una lista para almacenar datos procesados

    # Itera sobre las filas del DataFrame
    for _, row in df.iterrows():
        property_id = row['property_id']  # Extrae el 'property_id'

        # Itera sobre los elementos en la columna objetivo
        for elemento in row[columna_target]:
            if elemento is not None:
                try:
                    elemento_data = eval(elemento).copy()  # Convierte a diccionario y copia
                except:
                    elemento_data = elemento.copy()  # Si no se puede evaluar, copia el elemento
            else:
                elemento_data = {}

            elemento_data['property_id'] = property_id  # Agrega 'property_id' al diccionario
            lista_dics_datos.append(elemento_data)  # Agrega el diccionario a la lista de datos

    # Crea un nuevo DataFrame con los datos procesados
    df_limpio = pd.DataFrame(lista_dics_datos, columns=columnas)

    # Devuelve el nuevo DataFrame procesado
    return df_limpio

In [6]:
def unraveler(df, target_columns):
    """
    Desanida datos y combina los resultados en un DataFrame final.

    Parameters:
        df (DataFrame): DataFrame que contiene los datos.
        target_columns (list): Lista de nombres de columnas a desanidar.

    Returns:
        DataFrame: DataFrame desanidado.
    """
    dfs_temporales = []  # Inicializa una lista para almacenar DataFrames temporales

    # Itera sobre las columnas a desanidar
    for column in target_columns:
        df_grouped = df.groupby('property_id')[column].apply(list).reset_index()  # Agrupa y lista por 'property_id'
        columns = ['property_id']  # Inicializa la lista de columnas

        # Intenta obtener las claves de los diccionarios y agregarlas a las columnas
        try:
            columns += [x for x in eval(df_grouped[column][0][0]).keys()]
        except:
            columns += [x for x in (df_grouped[column][0][0]).keys()]

        # Aplica la función de procesamiento de datos para desanidar
        df_temporal = data_processor(df_grouped, columns, column)
        dfs_temporales.append(df_temporal)  # Agrega el DataFrame temporal a la lista

    # Combina los DataFrames temporales en uno final
    resultado = reduce(lambda left, right: pd.merge(left, right, on='property_id'), dfs_temporales)

    # Combina con el DataFrame original por 'property_id'
    resultado = pd.merge(resultado, df, on='property_id')

    # Devuelve el DataFrame final desanidado
    return resultado


In [9]:

# Definir función para limpiar y transformar los datos
def clean_and_transform_data():
    # Cargar el conjunto de datos desde un archivo CSV llamado 'Dataset_houses_for_sale.csv'
    houses_for_sale = pd.read_csv('Dataset_houses_for_sale.csv')
    # Columnas a eliminar del DataFrame 'houses_for_sale'
    columns_to_drop = ['source', 'permalink', 'other_listings', 'open_houses', 'branding', 'coming_soon_date',
                       'matterport', 'search_promotions', 'rent_to_own', 'products', 'virtual_tours', 'community',
                       'price_reduced_amount']
    # Eliminar las columnas especificadas del DataFrame 'houses_for_sale'
    houses_for_sale.drop(columns=columns_to_drop, inplace=True)
    # Agrupar el DataFrame 'houses_for_sale' por 'property_id' y convertir las descripciones a listas
    df = houses_for_sale.groupby('property_id')['description'].apply(list).reset_index()
    # Obtener las columnas 'property_id' y las claves de las descripciones de la primera fila
    columns = ['property_id']
    columns += [x for x in eval(df['description'][0][0]).keys()]
     # Procesar el DataFrame 'df' con las columnas obtenidas y la descripción
    df_temporal = data_processor(df, columns, 'description')
    # Combinar los DataFrames 'df_temporal' y 'houses_for_sale' en función de 'property_id'
    df_final = pd.merge(df_temporal, houses_for_sale, on='property_id')
    # Columnas que se eliminarán del DataFrame 'df_final'
    droppable_columns = ['last_update_date', 'description', 'lead_attributes', 'tax_record']
    # Eliminar las columnas especificadas del DataFrame 'df_final'
    df_final.drop(columns=droppable_columns, inplace=True)

In [11]:
# Función para extraer números de una cadena de texto
def extract_numbers(text):
    """
    Esta función utiliza expresiones regulares para encontrar números en una cadena de texto.

    Parameters:
        text (str): Cadena de texto que se va a analizar.

    Returns:
        str: Números encontrados en la cadena, separados por comas. Si no se encuentran números, retorna NaN.
    """
    numbers_found = re.findall(r'\d+\.\d+|\d+', str(text))  # Encuentra números en el texto usando una expresión regular
    if numbers_found:
        return ','.join(numbers_found)  # Devuelve los números encontrados separados por comas
    return np.nan  # Si no se encuentran números, devuelve NaN



In [13]:
# Aplicar la función extract_numbers a la columna 'baths_consolidated' en df_final
df_final['baths_consolidated'] = df_final['baths_consolidated'].apply(extract_numbers)

# Convertir los números extraídos a tipo de datos float
df_final['baths_consolidated'] = df_final['baths_consolidated'].apply(float)

NameError: name 'df_final' is not defined

In [14]:
# Función para convertir una cadena de texto en un objeto de fecha
def convert_date(text):
    """
    Esta función intenta convertir una cadena de texto en un objeto de fecha.

    Parameters:
        text (str): Cadena de texto que se intentará convertir a fecha.

    Returns:
        datetime: Objeto de fecha si la conversión es exitosa, de lo contrario, devuelve NaN.
    """
    try:
        return pd.to_datetime(text, errors='raise')  # Intenta convertir la cadena a fecha
    except ValueError:
        return np.nan  # Si hay un error en la conversión, devuelve NaN

In [15]:
# Aplicar la función convert_date a la columna 'sold_date' en df_final
df_final['sold_date'] = df_final['sold_date'].apply(convert_date)

# Filtrar filas donde 'sold_date' no es NaN y guardar en un archivo CSV
df_final[df_final['sold_date'].notna()].to_csv('Df_sold_homes_to_train.csv')

# Columnas que se eliminarán del DataFrame 'df_final'
columns_to_drop = ['sold_date', 'sold_price', 'name', 'sub_type']

# Eliminar las columnas especificadas del DataFrame 'df_final'
df_final.drop(columns=columns_to_drop, inplace=True)

# Convertir 'list_date' a una cadena de texto y mantener solo la fecha (parte antes de 'T')
df_final['list_date'] = (df_final['list_date'].astype(str).str.split('T')).str[0]

# Convertir 'list_date' a fecha usando la función convert_date
df_final['list_date'] = df_final['list_date'].apply(convert_date)

# Imprimir columnas y sus valores para la tercera fila del DataFrame 'df_final'
for column in df_final.columns:
    if type(df_final[column][2]) is str:
        print(column, df_final[column][2]+'\n')

# Mostrar las primeras filas de ciertas columnas del DataFrame 'df_final'
df_final[['primary_photo', 'tags', 'photos', 'flags', 'location']].head()

NameError: name 'df_final' is not defined

In [16]:
# Función para evaluar una expresión de forma segura
def safe_eval(expression):
    """
    Esta función intenta evaluar una expresión de forma segura utilizando eval().

    Parameters:
        expression (str): Expresión que se intentará evaluar.

    Returns:
        result: Resultado de la evaluación de la expresión o la expresión original si hay un error.
    """
    try:
        result = eval(expression)  # Intenta evaluar la expresión
        return result
    except Exception as e:
        if expression is np.nan:
            return expression  # Si la expresión es NaN, devuelve NaN
        else:
            print(f"Error en la evaluación: {e}, and was given {expression}")  # Imprime el error
            return expression  # Si hay un error, devuelve la expresión original

In [ ]:

# Aplicar la función safe_eval a la columna 'tags' en df_final
df_final['tags'] = df_final['tags'].apply(safe_eval)


In [17]:
# Función para obtener las etiquetas (tags) de una lista
def get_tags(lst):
    """
    Esta función toma una lista y devuelve las etiquetas presentes en ella.

    Parameters:
        lst (list): Lista que contiene etiquetas.

    Returns:
        list: Lista de etiquetas extraídas.
    """
    my_list = []
    try:
        for tag in lst:
            my_list.append(tag)
        return my_list
    except:
        return my_list

In [8]:
# Inicializar una lista para almacenar las etiquetas
tags = []

# Iterar sobre el DataFrame 'df_final' y obtener las etiquetas de la columna 'tags'
for index, row in df_final.iterrows():
    tag = get_tags(row['tags'])
    tags += tag

# Crear una lista de palabras y un texto uniendo las etiquetas
lista_palabras = tags
texto = " ".join(lista_palabras)


# Eliminar etiquetas relacionadas con 'garage', 'story' o 'stories' de la lista de etiquetas
Start_tags = 0
current_tags = 1
while Start_tags != current_tags:
    Start_tags = len(tags)
    for tag in tags:
        if 'garage' in tag or 'story' in tag or 'stories' in tag:
            tags.remove(tag)
    current_tags = len(tags)

# Contar y almacenar la cantidad de cada etiqueta única
tags_unicos = {}
for tag in tags:
    if tag not in tags_unicos.keys():
        tags_unicos[tag] = tags.count(tag)
    else:
        tags.remove(tag)

# Crear un DataFrame con las 50 etiquetas más comunes
top_50_tags = pd.DataFrame(tags_unicos.values(), tags_unicos.keys()).sort_values(by=0, ascending=False).head(50).rename(columns={0: 'count'})

# Lista de columnas a considerar
columns = ['flags', 'location']

# Aplicar la función unraveler para desenrollar las columnas en df_final
df_final = unraveler(df_final, columns)

# Eliminar columnas no necesarias del DataFrame df_final
df_final.drop(columns=['is_new_listing', 'is_pending', 'flags', 'location', "is_subdivision"], inplace=True)

# Identificar columnas relacionadas con baños y almacenarlas en bath_columns
bath_columns = []
for column in df_final.columns:
    if 'bath' in column:
        bath_columns.append(column)

# Iterar sobre las columnas relacionadas con baños y mostrar números mayores a 0 en filas con 'baths_consolidated' NaN
for column in bath_columns:
    for num in df_final[df_final['baths_consolidated'].isna() == True][column]:
        if num > 0:
            print(num)

# Remover 'baths_consolidated' de bath_columns
bath_columns.remove('baths_consolidated')

# Eliminar las columnas relacionadas con baños que no son 'baths_consolidated' del DataFrame df_final
df_final.drop(bath_columns, axis=1, inplace=True)

# Renombrar la columna 'baths_consolidated' a 'baths'
df_final = df_final.rename(columns={"baths_consolidated": "baths"})

# Imprimir ciertas columnas y sus valores para la tercera fila del DataFrame df_final
for column in df_final.columns:
    if type(df_final[column][2]) is str:
        print(f'{column}', df_final[column][2]+'\n')

# Definir una nueva lista de columnas a considerar
columns = ['address', "county"]

# Aplicar la función unraveler para desenrollar las nuevas columnas en df_final
df_final = unraveler(df_final, columns)

# Desenrollar la columna 'coordinate' en df_final
df_final = unraveler(df_final, ['coordinate'])

# Eliminar columnas no necesarias del DataFrame df_final
df_final.drop(columns=['coordinate', 'county', 'address'], inplace=True)

# Aplicar la función safe_eval a la columna 'photos' en df_final
df_final['photos'] = df_final['photos'].apply(safe_eval)

# Guardar el DataFrame limpio en un archivo CSV
df_final.to_csv('houses_for_sale_limpio.csv', index=False)


type single_family

primary_photo {'href': 'https://ap.rdcpix.com/fd492f6878f09cbe6eda231a8fcceb73l-m763238679s-w1024_h768.jpg'}

tags ['community_boat_facilities', 'community_golf', 'community_outdoor_space', 'community_security_features', 'den_or_office', 'energy_efficient', 'fireplace', 'golf_course_lot_or_frontage', 'golf_course_view', 'laundry_room', 'pets_allowed', 'recreation_facilities', 'view', 'single_story', 'garage_1_or_more', 'garage_2_or_more', 'garage_3_or_more', 'gourmet_kitchen', 'high_ceiling', 'wine_cellar', 'gated_community', 'clubhouse', 'golf_course', 'views', 'groundscare', 'security']

photos [{'href': 'https://ap.rdcpix.com/fd492f6878f09cbe6eda231a8fcceb73l-m763238679s-w1024_h768.jpg'}, {'href': 'https://ap.rdcpix.com/fd492f6878f09cbe6eda231a8fcceb73l-m3583859794s-w1024_h768.jpg'}]

flags {'is_new_construction': None, 'is_subdivision': None, 'is_plan': None, 'is_price_reduced': None, 'is_pending': None, 'is_foreclosure': None, 'is_new_listing': True, 'is_coming

KeyboardInterrupt: 